<a href="https://colab.research.google.com/github/gracelin0711/20250218/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

In [ ]:
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN_2'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "confirm":
          reply = TemplateMessage(
              alt_text="這是確認視窗",
              template = ConfirmTemplate(
                  text="你喜歡葬送的芙莉蓮嗎？",
                  actions=[
                      MessageAction(label="是", text="我超喜歡芙莉蓮!"),
                      MessageAction(label="否", text="其實我非常喜歡，只是我要傲驕的說不")
                      ]
                  )
              )
        elif action == 'carousel':
          carousel_template = CarouselTemplate(
              columns=[
                  CarouselColumn(
                      thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                      title='夏日大作戰',
                      text='細田守導演的日本科幻電影',
                      actions=[
                          URIAction(label='維基百科', uri='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg'),
                          MessageAction(label="投票", text="我投夏日大作戰一票")
                          ]
                      ),
                  CarouselColumn(
                      thumbnail_image_url='https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96_%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E4%B9%8B%E5%9F%8E',
                      title='魯邦三世 卡里奧斯特羅城',
                      text='宮崎駿導演的日本動畫動作冒險喜劇電影',
                      actions=[
                          URIAction(label='維基百科', uri='https://example.com/productB'),
                          MessageAction(label="投票", text="我投魯邦三世一票")
                          ]
                      )
                  ]
              )
          reply = TemplateMessage(
              alt_text="這是輪播視窗",
              template=carousel_template
              )
        else:
          reply = TextMessage(text='Please type"confirm"!')


        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6d34-34-23-84-5.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:42:07] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:42:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:44:37] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:44:46] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:45:00] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:45:07] "POST /callback HTTP/1.1" 200 -
